In [61]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [62]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.


### Buscar IDs dos times no Cartola

In [63]:
nomes_times = ['KP JUV.', 'Noah A 10', 'pura bucha /botafogo', 'Tatols Beants F.C', 'DM Studio', 'cartola scheuer', 'Texas Club 2025', 'Super Vasco f.c', 'TEAM LOPES 99', 'S.E.R. GRILLO',
               'TIGRE LEON', 'Fedato Futebol Clube', 'HS SPORTS F.C', 'BORGES CLIMA FUT F.C', 'SERGRILLO', 'Dom Camillo68', 'seralex', 'lsauer fc', 'ITAQUI F. C.', 'Cril Futebol Club']

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# ➕ Correções manuais
# ids_times["BORGES ITAQUI F.C."] = 3914981
# ids_times["Time do S.A.P.O"] = 12345678  # Substitua pelo ID real

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [64]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_C.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_C.xlsx


,Nome do Time,ID do Time,Link do Time
0,KP JUV.,18346776,https://cartola.globo.com/#!/time/18346776
1,Noah A 10,49960687,https://cartola.globo.com/#!/time/49960687
2,pura bucha /botafogo,18661583,https://cartola.globo.com/#!/time/18661583
3,Tatols Beants F.C,212042,https://cartola.globo.com/#!/time/212042
4,DM Studio,387186,https://cartola.globo.com/#!/time/387186
5,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
6,Texas Club 2025,1273719,https://cartola.globo.com/#!/time/1273719
7,Super Vasco f.c,13707047,https://cartola.globo.com/#!/time/13707047
8,TEAM LOPES 99,479510,https://cartola.globo.com/#!/time/479510
9,S.E.R. GRILLO,5823700,https://cartola.globo.com/#!/time/5823700


### Gerar o dicionário ID -> Nome do Time

In [65]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:20])

{18346776: 'KP JUV.',
 49960687: 'Noah A 10',
 18661583: 'pura bucha /botafogo',
 212042: 'Tatols Beants F.C',
 387186: 'DM Studio',
 3851966: 'cartola scheuer',
 1273719: 'Texas Club 2025',
 13707047: 'Super Vasco f.c',
 479510: 'TEAM LOPES 99',
 5823700: 'S.E.R. GRILLO',
 3424598: 'TIGRE LEON',
 18642587: 'Fedato Futebol Clube',
 17887202: 'HS SPORTS F.C',
 49852616: 'BORGES CLIMA FUT F.C',
 25811332: 'SERGRILLO',
 20696550: 'Dom Camillo68',
 29228373: 'seralex',
 44810918: 'lsauer fc',
 14369: 'ITAQUI F. C.',
 5664592: 'Cril Futebol Club'}

In [66]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [67]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7
KP JUV.,72.24,103.19,92.35,75.63,84.61,98.06,85.69
Noah A 10,85.15,75.29,94.62,116.71,80.18,121.36,113.49
pura bucha /botafogo,73.78,73.84,77.12,88.32,76.42,77.50,98.79
Tatols Beants F.C,66.60,73.32,99.55,94.87,83.03,86.06,105.71
DM Studio,72.55,81.69,82.67,106.62,73.53,86.06,133.81
cartola scheuer,73.09,86.39,75.10,77.36,80.92,98.30,95.79
Texas Club 2025,67.69,85.99,76.92,79.86,71.32,119.85,98.71
Super Vasco f.c,60.55,96.99,71.40,90.63,85.32,100.86,96.39
TEAM LOPES 99,74.09,73.19,68.42,109.06,87.27,106.06,127.76
S.E.R. GRILLO,68.48,81.12,53.84,60.93,72.33,95.32,86.09


### Função para definir a classificação dos times

In [68]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [69]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série C"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head(20))

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,TEAM LOPES 99,TIGRE LEON,13,14,Série C
1,1,2,Fedato Futebol Clube,Dom Camillo68,5,4,Série C
2,1,3,Texas Club 2025,S.E.R. GRILLO,16,10,Série C
3,1,4,HS SPORTS F.C,seralex,6,20,Série C
4,1,5,Noah A 10,Cril Futebol Club,9,2,Série C


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
8,Série C,Noah A 10,18,6,0,1,686.80,607.62,79.18,1
14,Série C,DM Studio,18,6,0,1,636.93,564.72,72.21,2
19,Série C,KP JUV.,18,6,0,1,611.77,542.78,68.99,3
10,Série C,cartola scheuer,15,5,0,2,586.95,535.58,51.37,4
0,Série C,TEAM LOPES 99,12,4,0,3,645.85,620.61,25.24,5
3,Série C,Dom Camillo68,12,4,0,3,640.77,607.75,33.02,6
12,Série C,Tatols Beants F.C,12,4,0,3,609.14,572.46,36.68,7
13,Série C,Super Vasco f.c,12,4,0,3,602.14,599.40,2.74,8
16,Série C,pura bucha /botafogo,12,4,0,3,565.77,553.10,12.67,9
4,Série C,Texas Club 2025,9,3,0,4,600.34,568.35,31.99,10


In [70]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [71]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_C.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_C.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_C.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_C.js'

In [72]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série C"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_C.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_C.js gerado com sucesso!")


✅ Arquivo confrontos_serie_C.js gerado com sucesso!


In [73]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

display(df_confrontos.head(10))

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,TEAM LOPES 99,TIGRE LEON,13,14
1,1,2,Fedato Futebol Clube,Dom Camillo68,5,4
2,1,3,Texas Club 2025,S.E.R. GRILLO,16,10
3,1,4,HS SPORTS F.C,seralex,6,20
4,1,5,Noah A 10,Cril Futebol Club,9,2
5,1,6,cartola scheuer,SERGRILLO,17,11
6,1,7,Tatols Beants F.C,Super Vasco f.c,15,12
7,1,8,DM Studio,lsauer fc,3,18
8,1,9,pura bucha /botafogo,ITAQUI F. C.,19,7
9,1,10,BORGES CLIMA FUT F.C,KP JUV.,1,8


In [74]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série C"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série C por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [75]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_C.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_C.js gerado com sucesso.")


✅ Arquivo resultados_serie_C.js gerado com sucesso.
